<font color='blue'>Whenever we build a model, there will be gaps between what a model predicts and what is observed in the sample. The differences between these values are known as the residuals of the model and can be used to check for some of the basic assumptions that go into the model. 

<font color='blue'>The key assumptions to check for are:
- <font color='blue'>Linear Fit</font>: The underlying relationship should be linear
- <font color='blue'>Homoscedastic</font>: The data should have no trend in the variance
- <font color='blue'>Independent and Identically Distributed</font>: The residuals of the regression should be independent and identically distributed (i.i.d.) and show no signs of serial correlation

<font color='blue'>We can use the residuals to help diagnose whether the relationship we have estimated is real or spurious.

Statistical error is a similar metric associated with regression analysis with one important difference: <font color='blue'>While residuals quantify the gap between a regression model predictions and the observed sample, statistical error is the difference between a regression model and the unobservable expected value.

We use residuals in an attempt to estimate this error.

In [ ]:
from statsmodels import regression
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import scipy.stats as stats

In [ ]:
def linreg(X,Y):
    # Running the linear regression
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    B0 = model.params[0]
    B1 = model.params[1]
    X = X[:, 1]
    # Return summary of the regression and plot results
    X2 = np.linspace(X.min(), X.max(), 100)
    Y_hat = X2 * B1 + B0
    plt.scatter(X, Y, alpha=1) # Plot the raw data
    plt.plot(X2, Y_hat, 'r', alpha=1);  # Add the regression line, colored in red
    plt.xlabel('X Value')
    plt.ylabel('Y Value')
    return model, B0, B1

In [ ]:
n = 50
X = np.random.randint(0, 100, n)
epsilon = np.random.normal(0, 1, n)

Y = 10 + 0.5 * X + epsilon

linreg(X,Y)[0];
print "Line of best fit: Y = {0} + {1}*X".format(linreg(X, Y)[1], linreg(X, Y)[2])

<font color='blue'>Having a model that fits every single observation that you have is a sure sign of overfitting.

<font color='blue'>The definition of a residual is the difference between what is observed in the sample and what is predicted by the regression. 

For any residual $r_i$, we express this as 
$r_i = Y_i - \hat{Y_i}$
, where $Y_i$ is the observed $Y$-value and $\hat{Y}_i$ is the predicted Y-value.

In [ ]:
model, B0, B1 = linreg(X,Y)

residuals = model.resid
plt.errorbar(X,Y,xerr=0,yerr=[residuals,0*residuals],linestyle="None",color='Green');

In [ ]:
residuals = model.resid
print residuals

<font color='blue'>Many of the assumptions that are necessary to have a valid linear regression model can be checked by identifying patterns in the residuals of that model.

With a <font color='blue'>residual plot</font>, we look at the predicted values of the model versus the residuals themselves. 

In [ ]:
plt.scatter(model.predict(), residuals);
plt.axhline(0, color='red')
plt.xlabel('Predicted Values');
plt.ylabel('Residuals');
plt.xlim([1,50]);

<font color='blue'>What we want is a fairly random distribution of residuals. The points should form no discernible pattern. This would indicate that a plain linear model is likely a good fit.

 <font color='blue'>If we see any sort of trend, this might indicate the presence of autocorrelation or heteroscedasticity in the model.

By looking for patterns in residual plots we can determine whether a linear model is appropriate in the first place. 

<font color='blue'>If the relationship is not a good fit for a linear model, the residual plot will show a distinct pattern

<font color='blue'>In general, a residual plot of a linear regression on a non-linear relationship will show bias and be asymmetrical with respect to residual = 0 line while a residual plot of a linear regression on a linear relationship will be generally symmetrical over the residual = 0 axis.

In [ ]:
n = 50
X = np.random.randint(0, 50, n)
epsilon = np.random.normal(0, 1, n)
Y_nonlinear = 10 - X**1.2 + epsilon

model = sm.OLS(Y_nonlinear, sm.add_constant(X)).fit()
B0, B1 = model.params
residuals = model.resid

print 'beta_0: ', B0
print 'beta_1: ', B1
plt.scatter(model.predict(), residuals);
plt.axhline(0, color='red')
plt.xlabel('Predicted Values');
plt.ylabel('Residuals');

One of the main assumptions behind a linear regression is that the underlying data has a constant variance. <font color='blue'>If there are some parts of the data with a variance different from another part the data is not appropriate for a linear regression.

Heteroscedasticity is a term that refers to data with non-constant variance, as opposed to homoscedasticity, when data has constant variance.

<font color='blue'>Significant heteroscedasticity invalidates linear regression results by biasing the standard error of the model. As a result, we can't trust the outcomes of significance tests and confidence intervals generated from the model and its parameters.

To avoid these consequences <font color='blue'>it is important to use residual plots to check for heteroscedasticity and adjust if necessary.

In [ ]:
n = 50
X = np.random.randint(0, 100, n)
epsilon = np.random.normal(0, 1, n)
Y_heteroscedastic = 100 + 2*X + epsilon*X

model = sm.OLS(Y_heteroscedastic, sm.add_constant(X)).fit()
B0, B1 = model.params
residuals = model.resid

plt.scatter(model.predict(), residuals);
plt.axhline(0, color='red')
plt.xlabel('Predicted Values');
plt.ylabel('Residuals');

<font color='blue'>A common way to test for the presence of heteroscedasticity is the Breusch-Pagan hypothesis test.

It's good to <font color='blue'>combine the qualitative analysis of a residual plot with the quantitative analysis of at least one hypothesis test.

We can add the <font color='blue'>White test </font>as well, but for now we will use only Breush-Pagan to test our relationship above. 

A function exists in the `statsmodels` package called `het_breushpagan` that simplifies the computation:

In [ ]:
breusch_pagan_p = smd.het_breushpagan(model.resid, model.model.exog)[1]
print breusch_pagan_p
if breusch_pagan_p > 0.05:
    print "The relationship is not heteroscedastic."
if breusch_pagan_p < 0.05:
    print "The relationship is heteroscedastic."

We set our confidence level at  α=0.05 , so a Breusch-Pagan p-value below  0.05  tells us that the relationship is heteroscedastic.

<font color='blue'>Using a hypothesis test bears the risk of a false positive or a false negative, which is why it can be good to confirm with additional tests if we are skeptical.

If, after creating a residual plot and conducting tests, <font color='blue'>you believe you have heteroscedasticity, there are a number of methods you can use to attempt to adjust for it. 

The three we will focus on are differences analysis, log transformations, and Box-Cox transformations.

<font color='blue'>A differences analysis involves looking at the first-order differences between adjacent values. With this, we are looking at the changes from period to period of an independent variable rather than looking directly at its values. 

 Often, by looking at the differences instead of the raw values, we can remove heteroscedasticity. <font color='blue'>We correct for it and can use the ensuing model on the differences.

In [ ]:
# Finding first-order differences in Y_heteroscedastic
Y_heteroscedastic_diff = np.diff(Y_heteroscedastic)

In [ ]:
model = sm.OLS(Y_heteroscedastic_diff, sm.add_constant(X[1:])).fit()
B0, B1 = model.params
residuals = model.resid

plt.scatter(model.predict(), residuals);
plt.axhline(0, color='red')
plt.xlabel('Predicted Values');
plt.ylabel('Residuals');

In [ ]:
breusch_pagan_p = smd.het_breushpagan(residuals, model.model.exog)[1]
print breusch_pagan_p
if breusch_pagan_p > 0.05:
    print "The relationship is not heteroscedastic."
if breusch_pagan_p < 0.05:
    print "The relationship is heteroscedastic."

This new regression was conducted on the differences between data, and therefore the regression output must be back-transformed to reach a prediction in the original scale. Since we regressed the differences, we can add our predicted difference onto the original data to get our estimate:*
$$\hat{Y_i} = Y_{i-1} + \hat{Y}_{diff}$$

 <font color='blue'>A log transformation will bring residuals closer together and ideally remove heteroscedasticity. In many (though not all) cases, a log transformation is sufficient in stabilizing the variance of a relationship.

In [ ]:
# Taking the log of the previous data Y_heteroscedastic and saving it in Y_heteroscedastic_log
Y_heteroscedastic_log = np.log(Y_heteroscedastic)

In [ ]:
model = sm.OLS(Y_heteroscedastic_log, sm.add_constant(X)).fit()
B0, B1 = model.params
residuals = model.resid

plt.scatter(model.predict(), residuals);
plt.axhline(0, color='red')
plt.xlabel('Predicted Values');
plt.ylabel('Residuals');

In [ ]:
# Running and interpreting a Breusch-Pagan test
breusch_pagan_p = smd.het_breushpagan(residuals, model.model.exog)[1]
print breusch_pagan_p
if breusch_pagan_p > 0.05:
    print "The relationship is not heteroscedastic."
if breusch_pagan_p < 0.05:
    print "The relationship is heteroscedastic."

This new regression was conducted on the log of the original data. This means the scale has been altered and the regression estimates will lie on this transformed scale. To bring the estimates back to the original scale, you must back-transform the values using the inverse of the log:*

$$\hat{Y} = e^{\log(\hat{Y})}$$

<font color='blue'>The Box-Cox transformation is a powerful method that will work on many types of heteroscedastic relationships.

The process works by testing all values of $\lambda$ within the range $[-5, 5]$ to see which makes the output of the following equation closest to being normally distributed:
$$
Y^{(\lambda)} = \begin{cases}
    \frac{Y^{\lambda}-1}{\lambda} & : \lambda \neq 0\\ \log{Y} & : \lambda = 0
\end{cases}
$$

The "best" $\lambda$ will be used to transform the series along the above function. Instead of having to do all of this manually, <font color='blue'>we can simply use the `scipy` function `boxcox`. We use this to adjust $Y$ and hopefully remove heteroscedasticity.

<font color='blue'>The Box-Cox transformation can only be used if all the data is positive* 

In [ ]:
# Finding a power transformation adjusted Y_heteroscedastic
Y_heteroscedastic_box_cox = stats.boxcox(Y_heteroscedastic)[0]

In [ ]:
model = sm.OLS(Y_heteroscedastic_box_cox, sm.add_constant(X)).fit()
B0, B1 = model.params
residuals = model.resid

plt.scatter(model.predict(), residuals);
plt.axhline(0, color='red')
plt.xlabel('Predicted Values');
plt.ylabel('Residuals');

In [ ]:
# Running and interpreting a Breusch-Pagan test
breusch_pagan_p = smd.het_breushpagan(residuals, model.model.exog)[1]
print breusch_pagan_p
if breusch_pagan_p > 0.05:
    print "The relationship is not heteroscedastic."
if breusch_pagan_p < 0.05:
    print "The relationship is heteroscedastic."

However, because the data was power transformed, the regression estimates will be on a different scale than the original data. This is why it is important to remember to back-transform results using the inverse of the Box-Cox function:*

$$\hat{Y} = (Y^{(\lambda)}\lambda + 1)^{1/\lambda}$$


Another assumption behind linear regressions is that the residuals are not autocorrelated.

<font color='blue'>A series is autocorrelated when it is correlated with a delayed version of itself. 

<font color='blue'>In finance, stock prices are usually autocorrelated while stock returns are independent from one day to the next. 

We represent a time dependency on previous values like so:
$Y_i = Y_{i-1} + \epsilon$

<font color='blue'>If the residuals of a model are autocorrelated, you will be able to make predictions about adjacent residuals.

In [ ]:
n = 50
X = np.linspace(0, n, n)
Y_autocorrelated = np.zeros(n)
Y_autocorrelated[0] = 50
for t in range(1, n):
    Y_autocorrelated[t] = Y_autocorrelated[t-1] + np.random.normal(0, 1) 

# Regressing X and Y_autocorrelated
model = sm.OLS(Y_autocorrelated, sm.add_constant(X)).fit()
B0, B1 = model.params
residuals = model.resid

plt.scatter(model.predict(), residuals);
plt.axhline(0, color='red')
plt.xlabel('Predicted Values');
plt.ylabel('Residuals');

<font color='blue'>A Ljung-Box test is used to detect autocorrelation in a time series. The Ljung-Box test examines autocorrelation at all lag intervals below a specified maximum and returns arrays containing the outputs for every tested lag interval.

<font color='blue'>Let's use the `acorr_ljungbox` function in `statsmodels` to test for autocorrelation in the residuals of our above model. </font>We use a max lag interval of $10$, and see if any of the lags have significant autocorrelation:

In [ ]:
ljung_box = smd.acorr_ljungbox(residuals, lags = 10)
print "Lagrange Multiplier Statistics:", ljung_box[0]
print "\nP-values:", ljung_box[1], "\n"

if any(ljung_box[1] < 0.05):
    print "The residuals are autocorrelated."
else:
    print "The residuals are not autocorrelated."

<font color='blue'>We can adjust for autocorrelation in many of the same ways that we adjust for heteroscedasticity.

In [ ]:
# Finding first-order differences in Y_autocorrelated
Y_autocorrelated_diff = np.diff(Y_autocorrelated)

In [ ]:
model = sm.OLS(Y_autocorrelated_diff, sm.add_constant(X[1:])).fit()
B0, B1 = model.params
residuals = model.resid

plt.scatter(model.predict(), residuals);
plt.axhline(0, color='red')
plt.xlabel('Predicted Values');
plt.ylabel('Residuals');

# Running and interpreting a Ljung-Box test
ljung_box = smd.acorr_ljungbox(residuals, lags = 10)
print "P-values:", ljung_box[1], "\n"

if any(ljung_box[1] < 0.05):
    print "The residuals are autocorrelated."
else:
    print "The residuals are not autocorrelated."

We can also perform a log transformation, if we so choose.

Let's calculate the market beta between TSLA and SPY using a simple linear regression, and then conduct a residual analysis on the regression to ensure the validity of our results. 

To regress TSLA and SPY, we will focus on their returns, not their price, and set SPY returns as our independent variable and TSLA returns as our outcome variable. 

The regression will give us a line of best fit: 
$\hat{r_{TSLA}} = \hat{\beta_0} + \hat{\beta_1}r_{SPY}$

The slope of the regression line $\hat{\beta_1}$ will represent our market beta, as for every $r$ percent change in the returns of SPY, the predicted returns of TSLA will change by $\hat{\beta_1}$.

In [ ]:
start = '2014-01-01'
end = '2015-01-01'
asset = get_pricing('TSLA', fields='price', start_date=start, end_date=end)
benchmark = get_pricing('SPY', fields='price', start_date=start, end_date=end)

# We have to take the percent changes to get to returns
# Get rid of the first (0th) element because it is NAN
r_a = asset.pct_change()[1:].values
r_b = benchmark.pct_change()[1:].values

# Regressing the benchmark b and asset a
r_b = sm.add_constant(r_b)
model = sm.OLS(r_a, r_b).fit()
r_b = r_b[:, 1]
B0, B1 = model.params

# Plotting the regression
A_hat = (B1*r_b + B0)
plt.scatter(r_b, r_a, alpha=1) # Plot the raw data
plt.plot(r_b, A_hat, 'r', alpha=1);  # Add the regression line, colored in red
plt.xlabel('TSLA Returns')
plt.ylabel('SPY Returns')

# Print our result
print "Estimated TSLA Beta:", B1

# Calculating the residuals
residuals = model.resid

Now that we have the regression results and residuals, we can conduct our residual analysis.

In [ ]:
plt.scatter(model.predict(), residuals);
plt.axhline(0, color='red')
plt.xlabel('TSLA Returns');
plt.ylabel('Residuals');

In [ ]:
bp_test = smd.het_breushpagan(residuals, model.model.exog)

print "Lagrange Multiplier Statistic:", bp_test[0]
print "P-value:", bp_test[1]
print "f-value:", bp_test[2]
print "f_p-value:", bp_test[3], "\n"
if bp_test[1] > 0.05:
    print "The relationship is not heteroscedastic."
if bp_test[1] < 0.05:
    print "The relationship is heteroscedastic."

Because the P-value is greater than 0.05, we do not have enough evidence to reject the null hypothesis that the relationship is homoscedastic. This result matches up with our qualitative conclusion.

In [ ]:
ljung_box = smd.acorr_ljungbox(r_a)
print "P-Values:", ljung_box[1], "\n"
if any(ljung_box[1] < 0.05):
    print "The residuals are autocorrelated."
else:
    print "The residuals are not autocorrelated."

Because the Ljung-Box test yielded p-values above 0.05 for all lags, we can conclude that the residuals are not autocorrelated. This result matches up with our qualitative conclusion.